<a href="https://colab.research.google.com/github/takatakamanbou/AdvML/blob/2024/AdvML2024_reportB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AdvML reportB

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/AdvML/AdvML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?AdvML)




----
## 準備
----


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# scikit-learn のいろいろ
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

---
### MNIST

この課題では，MNIST と呼ばれるデータセットを用いる．手書き数字画像を 0 から 9 の 10 クラスに分類する問題のためのものである．


#### データの入手

In [ ]:
# MNIST データセットの入手
Xraw, yraw = fetch_openml('mnist_784', version=1, parser='auto', return_X_y=True, as_frame=False)
Xall = Xraw / 255.0     # 画素値が [0, 255] の整数値なので [0, 1] の浮動小数点数値に変換
yall = yraw.astype(int) # クラスラベル．0 から 9 の整数値
print(Xall.shape, yall.shape)

In [ ]:
# 最初の20枚を可視化
nrow, ncol = 2, 10
fig, ax = plt.subplots(nrow, ncol, figsize=(10, 2))
for i in range(nrow):
    for j in range(ncol):
        img = Xall[i*ncol + j, ::].reshape((28, 28))
        ax[i, j].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
        ax[i, j].axis('off')
        ax[i, j].set_title(yall[i*ncol + j])

fig.tight_layout()
plt.show()

上記の画像の上についている数字は，その画像に対応するクラスラベルである．
ひとつの画像は 28 x 28 画素のグレイスケール画像であるため，$28\times 28 = 784$ 次元ベクトルとして扱う．

#### 学習データとテストデータの分割

MNIST のデータは全部で7万件ある．多くて実験が大変（学習に時間がかかる等）なので，最初の2万件を取り出し，[sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) を使って 16000 件の学習データと 4000 件のテストデータに分ける．

In [ ]:
yall[:20000].shape

In [ ]:
# 学習データとテストデータの分割
Xsub, ysub = Xall[:20000, :], yall[:20000]
XL, XT, yL, yT = train_test_split(Xsub, ysub, test_size=4000, random_state=4649, stratify=ysub)
print(XL.shape, yL.shape)
print(XT.shape, yT.shape)

---
## ロジスティック回帰
---


次のコードセルを実行すると，`XL` と `yL` を学習データとしてロジスティック回帰モデルの学習を行い，学習データおよびテストデータ（`XT` と `yT`）に対する識別率（正答したデータの割合）を算出することができる．

`%%time` は，このコードセルの実行に要した時間を表示させる指示である．`CPU times` の 3 つはいわゆる CPU 実行時間，`Wall time` が実時間．この notebook の実験では，`Wall time` に注目すればよい．

cf. [sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)


In [ ]:
%%time

# 学習
model = LogisticRegression()
model.fit(XL, yL)

# 学習データのクラスを予測して正解数を算出
yL_pred = model.predict(XL)
ncL = np.sum(yL_pred == yL)

# テストデータのクラスを予測して正解数を算出
yT_pred = model.predict(XT)
ncT = np.sum(yT_pred == yT)

# print
print(f'学習データ: {ncL/len(XL)}  テストデータ: {ncT/len(XT)}')

結果をメモしておくこと（この notebook に書くなら，「結果と考察」のセクションへ）．

---
## $k$-近傍法
---

同じデータに対して，$k$-近傍法を適用してみよう．
$k$-近傍法については，2024年度「機械学習I」第3回（ https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2024#ex03 ）参照．


以下のセルに，$k$-近傍法でロジスティック回帰と同様の実験を行うコードを書きなさい．
[sklearn.neighbors.KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) を使おう．

$k$-近傍法では，$k$ がハイパーパラメータとなる．$k$ をいろいろ変えて実行し，結果をメモしておくこと（この notebook に書くなら，「結果と考察」のセクションへ）．

---
## 階層型ニューラルネットの実験
---

以下のセルに，階層型ニューラルネットを用いて同様の実験を行うコードを書きなさい．

ただし...
- [sklearn.neural_network.MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) を使おう．
- 階層型ニューラルネットには様々なハイパーパラメータがあるが，ここでは，層の数とニューロン数のみを変えてみることにしよう．
- [sklearn.neural_network.MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) を見ると，`early_stopping` というオプションがある．
    - その値はデフォルトでは `False` であるが，`True` にして実験しよう．
    - どういう効果があるか，リンク先のドキュメントを読んで理解しよう．
    - あわせて，`verbose` も `True` にして実行しよう．
- 層の数やニューロン数が同じ条件のニューラルネットワークでも，パラメータの初期値が異なれば，違う学習結果が得られる．同じ条件でも複数回実行することが望ましい．


層数やニューロン数をいろいろ変えて実行し，結果をメモしておくこと（この notebook に書くなら，「結果と考察」のセクションへ）．

興味があれば，`early_stopping` のオンオフで結果がどのように違うかも調べてみるとよい．

---
## 結果と考察
---

以下に，上記で行った各種実験の結果とそれに対する考察を記しなさい．
学習手法ごとにその中でハイパーパラメータを変えるとどう変化したかを述べるだけではなく，学習手法を変えると結果がどう変化したかも考察すること．

注意:

- 結果を記す際には，コードセルを実行して得られる出力を貼り付けても構いませんが，不必要なものまで長々と貼り付けるのではなく，必要な情報を取捨選択し整理してください．
- Colab notebook には，Markdown 形式で文書を書けます．
セクションごとに見出しを付ける，箇条書きする，表を作る等，書き方を工夫したい場合は，「Markdown」で検索してみるとよいでしょう．